In [1]:
import pandas as pd
import os

In [2]:
#This will have to change after we set it up to pull from Drive
datapath =  os.getcwd() + "/data/"

In [3]:
query = pd.read_csv(datapath + "barcode13-query.csv")
hits = pd.read_csv(datapath + "barcode13-hits.csv")

In [4]:
len(query.columns)

31

In [5]:
len(hits.columns)

38

In [6]:
query.columns

Index(['Name', '# Source Sequences', '% GC', '% HQ', '% Identical Sites',
       '% LQ', '% MQ', '% Pairwise Identity', 'Ambiguities', 'At least Q20',
       'At least Q30', 'At least Q40', 'Bin', 'Created', 'Created Date',
       'Description', 'Failed Binning Fields', 'Free end gaps',
       'Mean Coverage', 'Modified', 'Molecular Weight (kDa)', 'Molecule Type',
       'Post-Trim Length', 'Rough Temperature (°C)', 'Sample', 'Sequence',
       'Sequence Length', 'Sequence List Name', 'Size', 'Topology', 'URN'],
      dtype='object')

In [10]:
dropped_query = query.dropna(axis=1,how='all')
dropped_query.columns

Index(['Name', '# Source Sequences', '% GC', '% HQ', '% Identical Sites',
       '% LQ', '% MQ', '% Pairwise Identity', 'Ambiguities', 'At least Q20',
       'At least Q30', 'At least Q40', 'Bin', 'Created', 'Created Date',
       'Description', 'Failed Binning Fields', 'Free end gaps',
       'Mean Coverage', 'Modified', 'Molecular Weight (kDa)', 'Molecule Type',
       'Post-Trim Length', 'Rough Temperature (°C)', 'Sample', 'Sequence',
       'Sequence Length', 'Size', 'Topology', 'URN'],
      dtype='object')

In [11]:
hits.columns

Index(['Name', '# Nucleotides', '# Sequences', '% GC', '% Identical Sites',
       '% Pairwise Identity', 'Accession', 'Bit-Score', 'Created',
       'Created Date', 'Database', 'Description', 'E Value', 'Free end gaps',
       'Grade', 'Hit end', 'Hit start', 'Max Sequence Length', 'Mean Coverage',
       'Min Sequence Length', 'Modified', 'Molecular Weight (kDa)',
       'Molecule Type', 'Organism', 'Original Query Frame', 'Query',
       'Query Id', 'Query coverage', 'Query end', 'Query start',
       'Ref Seq Index', 'Ref Seq Length', 'Ref Seq Name',
       'Rough Temperature (°C)', 'Sequence', 'Sequence Length', 'Topology',
       'URN'],
      dtype='object')

In [13]:
dropped_hits = hits.dropna(axis=1,how='all')
len(dropped_hits.columns)

36

In [14]:
dropped_hits.columns

Index(['Name', '# Nucleotides', '# Sequences', '% GC', '% Identical Sites',
       '% Pairwise Identity', 'Accession', 'Bit-Score', 'Created',
       'Created Date', 'Database', 'Description', 'E Value', 'Free end gaps',
       'Grade', 'Hit end', 'Hit start', 'Max Sequence Length', 'Mean Coverage',
       'Min Sequence Length', 'Modified', 'Molecular Weight (kDa)',
       'Molecule Type', 'Organism', 'Original Query Frame', 'Query',
       'Query coverage', 'Query end', 'Query start', 'Ref Seq Index',
       'Ref Seq Length', 'Rough Temperature (°C)', 'Sequence',
       'Sequence Length', 'Topology', 'URN'],
      dtype='object')

In [13]:
assert all(hits.columns == query.columns), "Columns in hits and query do not match"

ValueError: Lengths must match to compare

In [14]:
def clean_query_name(row):
    name = row['Name']
    name = name.split()
    for item in name:
        if "=" in item:
            item = item.split("=")
            row[item[0]] = item[1]

    return row


In [15]:
query = query.apply(clean_query_name, axis=1)
hits = hits.apply(clean_query_name, axis=1)

assert all(hits.columns == query.columns), "Columns in hits and query do not match"

ValueError: Lengths must match to compare

In [54]:
(query.isnull().sum() / query.shape[0])[query.isnull().sum() / query.shape[0] > 0.5]

# Source Sequences     0.995798
% Identical Sites      0.995798
% Pairwise Identity    0.995798
Description            0.995798
Free end gaps          0.995798
Mean Coverage          0.995798
Sample                 0.995798
Sequence List Name     1.000000
dtype: float64

In [55]:
(hits.isnull().sum() / hits.shape[0])[hits.isnull().sum() / hits.shape[0] > 0.5]

# Source Sequences           0.999321
% GC                         0.997283
% HQ                         0.997283
% Identical Sites            0.999321
% LQ                         0.997283
% MQ                         0.997283
% Pairwise Identity          0.999321
Ambiguities                  0.997283
At least Q20                 0.997283
At least Q30                 0.997283
At least Q40                 0.997283
Bin                          0.997283
Created                      0.997283
Failed Binning Fields        0.997283
Free end gaps                0.999321
Mean Coverage                0.999321
Modified                     0.996603
Molecular Weight (kDa)       0.997283
Molecule Type                0.997283
Post-Trim Length             0.997283
Rough Temperature (°C)       0.997283
Sample                       0.999321
Sequence Length              0.997283
Size                         0.997283
Topology                     0.996603
URN                          1.000000
barcode     

In [56]:
query["% Pairwise Identity"].notnull().sum()

np.int64(4)

In [57]:
filtered_hits = pd.read_csv(datapath + "barcode13-filtered-hits.csv")


In [61]:
filtered_hits = filtered_hits.apply(clean_query_name, axis=1)

In [62]:
filtered_hits.columns

Index(['# Source Sequences', '% Identical Sites', '% Pairwise Identity',
       'Created Date', 'Description', 'Free end gaps', 'Mean Coverage',
       'Modified', 'Name', 'Sample', 'Sequence', 'Sequence List Name',
       'Topology', 'URN', 'barcode', 'barcode_alias',
       'basecall_model_version_id', 'ch', 'flow_cell_id', 'parent_read_id',
       'protocol_group_id', 'runid', 'sample_id', 'start_time'],
      dtype='object')

In [58]:
hits[hits["parent_read_id"] != filtered_hits["parent_read_id"]]

KeyError: 'parent_read_id'

# Number of Unique Organisms Found

In [ ]:
hits.